<a href="https://colab.research.google.com/github/drigger091/CSV_CHATBOT/blob/main/Cohere_LLM_CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are using COHERE LLM CHATBOT to chat and do a QA on a provided text

In [19]:
  import cohere
  import warnings
  from annoy import AnnoyIndex
  import numpy as np
  import pandas as pd


In [20]:
text = """

India boasts a diverse coastline that stretches over 7,500 kilometers along the Arabian Sea, the Bay of Bengal, and the Indian Ocean,
offering a plethora of stunning beaches that cater to every taste and preference.
From secluded coves and pristine stretches of sand to vibrant beach towns bustling with
 life, India's beaches are a haven for travelers seeking sun, sea, and relaxation.

One of the most iconic beach destinations in India is Goa, renowned for its
golden sands, crystal-clear waters, and vibrant beach culture. From the lively
shores of Baga and Calangute to the tranquil beaches of Palolem and Agonda,
Goa offers a diverse array of beach experiences, whether you're looking to soak up
the sun, indulge in water sports, or party until dawn at one of its famous beach
shacks and nightclubs.

Further south along the western coast lies the picturesque state of Kerala,
home to some of India's most scenic beaches. The beaches of Kovalam, Varkala,
and Marari are renowned for their natural beauty, with palm-fringed shores,
gentle waves, and breathtaking sunsets that attract visitors from around the world.
Kerala's beaches also offer opportunities for cultural exploration, with nearby
attractions such as historic lighthouses, traditional fishing villages, and
Ayurvedic spas.
On the eastern coast, the state of Tamil Nadu boasts a wealth of
stunning beaches that cater to both leisure travelers and pilgrims alike.
Marina Beach in Chennai, one of the longest urban beaches in the world, is a
popular spot for locals and tourists to unwind and enjoy recreational activities such as kite flying and beach volleyball. Further south, the beach towns of Mahabalipuram and Pondicherry offer a mix of historical landmarks, coastal charm, and water sports, making them ideal destinations for a beach getaway with a cultural twist.

In the Andaman and Nicobar Islands, located in the Bay of Bengal,
 pristine beaches await those seeking a secluded tropical paradise.
 Radhanagar Beach on Havelock Island is consistently ranked among the
 best beaches in Asia, with its powdery white sands, azure waters, and
 thriving coral reefs ideal for snorkeling and scuba diving. Other highlights
 include Elephant Beach, known for its vibrant marine life, and Neil Island,
 where travelers can relax amidst coconut groves and turquoise lagoons.

In addition to these well-known destinations, India is dotted with
countless hidden gems waiting to be discovered along its coastline.
 Whether you're seeking adventure, relaxation, or simply a moment of
 tranquility by the sea, India's beaches offer something for everyone,
 making them an irresistible destination for beach lovers
 and nature enthusiasts alike."""

In [21]:
# prepocessing the text
try:
    texts = text.split("\n\n")
    texts = np.array([t.strip(r'\n') for t in texts if t])
except:
    print("Error preprocessing the text")
    exit()

In [22]:


# cohere API connection

try:
    co = cohere.Client("")

except Exception as e:
    print(f"Error connecting to cohere:{e}")
    exit



In [23]:
# embedding them
try:
    response =co.embed(texts=texts.tolist()).embeddings
    embeds = np.array(response)
except Exception as e:
    print(f"Error getting embeddings from Cohere:{e}")
    exit()

In [24]:
# Create a search index and pasing the size of embedding

search_index = AnnoyIndex(embeds.shape[1],'angular')
# adding vectors to search index
for i in range(len(embeds)):
  search_index.add_item(i,embeds[i])

search_index.build(10)
search_index.save("test.ann")

True

In [25]:
def search_text(query):
  try:
    query_embed =co.embed(texts=[query]).embeddings
    similar_items_ids = search_index.get_nns_by_vector(query_embed[0],
                                                       10,
                                                       include_distances=True)
    search_results = texts[similar_items_ids[0]]
    return search_results
  except Exception as e:
    print(f"Error during Search:{e}")


In [30]:
# creating the LLM chatbot
def ask_LLM(Question,num_generations =1):
        try:
            results = search_text(Question)
            context = results[0]
            prompt = f"""
More information about India beaches at India.com {context}
Question: {Question}
Extract answer from the given text if the text doesnt contain the answer reply the answer is not available"""

            prediction = co.generate(prompt= prompt,
                                     max_tokens = 70,
                                     model ="command-nightly",temperature = 0.4,
                                     num_generations = num_generations)
            return prediction.generations
        except Exception as e:
            print(f"Error occured during LLM call: {e}")
            return []



# Trying the chatbot demo

In [37]:

Q = input("Enter your question:")
answer_text = ask_LLM(Q)[0].text
print(answer_text)


Enter your question:Coastline length?
7,500 kilometers


In [40]:

Q = input("Enter your question:")
answer_text = ask_LLM(Q)[0].text
print(answer_text)


Enter your question:What kinds of activities can visitors enjoy at Goa's beaches?
Soak up the sun, indulge in water sports, or party at beach shacks and nightclubs.


In [41]:

Q = input("Enter your question:")
answer_text = ask_LLM(Q)[0].text
print(answer_text)


Enter your question:What recreational activities are popular at Marina Beach in Chennai?
Kite flying and beach volleyball.


In [45]:

Q = input("Enter your question:")
answer_text = ask_LLM(Q)[0].text
print(answer_text)


Enter your question:Can you name some of the bodies of water that border India's coastline?
Arabian Sea, Bay of Bengal, and the Indian Ocean.


In [44]:

Q = input("Enter your question:")
answer_text = ask_LLM(Q)[0].text
print(answer_text)


Enter your question:What is the tallest mountain in the world?
The answer is not available.
